In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [4]:
# Assignment 4 - Hypothesis Testing
#This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

#Definitions:
# * A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
# * A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
# * A _recession bottom_ is the quarter within a recession which had the lowest GDP.
# * A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

# **Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

# The following data files are available for this assignment:
# * From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
# * From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
# * From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

# Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [5]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [6]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan","Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State","RegionName"]  )'''
    with open('university_towns.txt') as file:
        data = []
        for line in file:
            data.append(line[:-1])
    state_town = []
    for line in data:
        if line[-6:] == '[edit]':
            state = line[:-6]
        elif '(' in line:
            town = line[:line.index('(')-1]
            state_town.append([state,town])
        else:
            town = line
            state_town.append([state,town])
    state_college_df = pd.DataFrame(state_town,columns = ['State','RegionName'])
    return state_college_df
        

get_list_of_university_towns()

FileNotFoundError: [Errno 2] No such file or directory: 'university_towns.txt'

In [7]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP_over_time = pd.read_excel("gdplev.xls", header = None, skiprows = 220)
    GDP_over_time = GDP_over_time[[4, 5]]
    GDP_over_time.columns = ["Quarterly", "GDP in billions of chained 2009 dollars"]
    values = GDP_over_time["GDP in billions of chained 2009 dollars"]
    indexes = []
    for i in range(1, len(values)-1):
        if  values[i] > values[i+1] and values[i+1] > values[i+2]:
            indexes.append(i)

    return GDP_over_time["Quarterly"][indexes[0]]
get_recession_start()


FileNotFoundError: [Errno 2] No such file or directory: 'gdplev.xls'

In [8]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    GDP_over_time = pd.read_excel("gdplev.xls", header = None, skiprows = 220)
    GDP_over_time = GDP_over_time[[4, 5]]
    GDP_over_time.columns = ["Quarterly", "GDP in billions of chained 2009 dollars"]
    values = GDP_over_time["GDP in billions of chained 2009 dollars"]
    indexes = []
    for i in range(2, len(values)-2):
        if   values [i-2]> values[i-1] < values[i] < values[i+1] and values[i+1] < values[i+2] :
            indexes.append(i)
    ans = GDP_over_time["Quarterly"][indexes[0]+1]
    return ans
get_recession_end()       
    

FileNotFoundError: [Errno 2] No such file or directory: 'gdplev.xls'

In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    q_start = get_recession_start()
    q_end = get_recession_end()

    GDP_over_time = pd.read_excel("gdplev.xls", header = None, skiprows = 220)
    GDP_over_time = GDP_over_time[[4, 5]]
    GDP_over_time.columns = ["Quarterly", "GDP in billions of chained 2009 dollars"]
    #data = GDP_over_time.loc[q_start:q_end,:]
    index1 = GDP_over_time[GDP_over_time["Quarterly"] == q_start].index.values[0]
    index2 = GDP_over_time[GDP_over_time["Quarterly"] == q_end].index.values[0]
    data = GDP_over_time[index1:index2+1].copy()
    data.sort_values("GDP in billions of chained 2009 dollars", axis = 0, inplace = True)

    return data["Quarterly"].iloc[0]
get_recession_bottom()


FileNotFoundError: [Errno 2] No such file or directory: 'gdplev.xls'

In [10]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting housing_dataframe should have 67 columns, and 10,730 rows.
    '''
    housing_data = pd.read_csv("City_Zhvi_AllHomes.csv")

    # changing the State code into State_name.
    housing_data["State"].replace({'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}, inplace = True)

    # dropping unneccesory columns.
    housing_data.drop(["RegionID", "Metro", "CountyName", "SizeRank"], axis = 1, inplace = True)
 

    # Sorting the State column and then doing MultiIndexing using State and RegionName.
    housing_data.sort_values("State", axis = 0, inplace = True)
    housing_data.set_index(["State", "RegionName"], inplace = True)

    # dropping unneccesory columns
    housing_data.drop(housing_data.iloc[:,:45], axis = 1, inplace = True)

    # getting mean values for a Quarter of three month for all the months.
    for i in range(0, len(housing_data.columns), 3):
        housing_data[i] = pd.DataFrame(housing_data.iloc[:,i:i+3].mean(axis = 1))

    # dropping the original one month housing_data for each year's month.
    housing_data.drop(housing_data.iloc[:,:200], axis =1, inplace = True)

    #renaming each column to corrosponding Quarter.
    housing_data.rename(columns = {0 : '2000q1', 3 : '2000q2', 6 : '2000q3', 9 : '2000q4',
                                  12 : '2001q1', 15 : '2001q2', 18 : '2001q3', 21 : '2001q4',
                                  24 : '2002q1', 27 : '2002q2', 30 : '2002q3', 33 : '2002q4',
                                  36 : '2003q1', 39 : '2003q2', 42 : '2003q3', 45 : '2003q4',
                                  48 : '2004q1', 51 : '2004q2', 54 : '2004q3', 57 : '2004q4',
                                  60 : '2005q1', 63 : '2005q2', 66 : '2005q3', 69 : '2005q4',
                                  72 : '2006q1', 75 : '2006q2', 78 : '2006q3', 81 : '2006q4',
                                  84 : '2007q1', 87 : '2007q2', 90 : '2007q3', 93 : '2007q4',
                                  96 : '2008q1', 99 : '2008q2', 102 : '2008q3', 105 : '2008q4',
                                  108 : '2009q1', 111 : '2009q2', 114 : '2009q3', 117 : '2009q4',
                                  120 : '2010q1', 123 : '2010q2', 126 : '2010q3', 129 : '2010q4',
                                  132 : '2011q1', 135 : '2011q2', 138 : '2011q3', 141 : '2011q4',
                                  144 : '2012q1', 147 : '2012q2', 150 : '2012q3', 153 : '2012q4',
                                  156 : '2013q1', 159 : '2013q2', 162 : '2013q3', 165 : '2013q4',
                                  168 : '2014q1', 171 : '2014q2', 174 : '2014q3', 177 : '2014q4',
                                  180 : '2015q1', 183 : '2015q2', 186 : '2015q3', 189 : '2015q4',
                                  192 : '2016q1', 195 : '2016q2', 198 : '2016q3'}, inplace = True)
    
    return housing_data
convert_housing_data_to_quarters()



FileNotFoundError: [Errno 2] File City_Zhvi_AllHomes.csv does not exist: 'City_Zhvi_AllHomes.csv'

In [11]:
def run_ttest():
#     '''First creates new data showing the decline or growth of housing prices
#     between the recession start and the recession bottom. Then runs a ttest
#     comparing the university town values to the non-university towns values, 
#     return whether the alternative hypothesis (that the two groups are the same)
#     is true or not as well as the p-value of the confidence. 
    
#     Return the tuple (different, p, better) where different=True if the t-test is
#     True at a p<0.01 (we reject the null hypothesis), or different=False if 
#     otherwise (we cannot reject the null hypothesis). The variable p should
#     be equal to the exact p value returned from scipy.stats.ttest_ind(). The
#     value for better should be either "university town" or "non-university town"
#     depending on which has a lower mean price ratio (which is equivilent to a
#     reduced market loss).'''
    # Getting the dataframe containing housing data.
    housing_data = convert_housing_data_to_quarters()

    # Getting university_town dataframe.
    university_data = get_list_of_university_towns()


    # Getting only the data in range on res_start and res_bottom from housing_data.  
    sample_data = housing_data[['2008q2', '2009q2']]

    ## Resetting Index of the data so we can split this into two dataframe, one containing all the university_town.
    sample_data.reset_index(inplace = True)

    ### Method 1 for splitting sample_data.
    ##### Using merge method.
    combined = sample_data.merge(university_data, how = 'outer', indicator = True)
    # we got the combined data of sample_data and university_data.
    # we can now get three different set of dataframe based on different masking("_merge")
    # but we need only two dataframes.
    university_towns = combined[combined['_merge'] == 'both']
    non_university_towns = combined[combined['_merge'] == 'left_only']
    university_towns.drop('_merge', axis = 1, inplace = True)
    non_university_towns.drop('_merge', axis = 1, inplace = True)
    non_university_towns["price ratio"] = non_university_towns["2008q2"]/non_university_towns["2009q2"]
    university_towns['price ratio'] = university_towns['2008q2']/university_towns['2009q2']
    non_university_towns.dropna(inplace = True)
    university_towns.dropna(inplace = True)                                              
    test = ttest_ind(university_towns['price ratio'], non_university_towns['price ratio'])
    res_null_hypo = None
    lower_mean_price_ratio = None
    if test[1] < 0.01:
        res_null_hypo = True
    else:
        res_null_hypo = False
    if test[0] < 0:
        lower_mean_price_ratio = "university town"
    else:
        lower_mean_price_ratio = "non-university town"
    return ( res_null_hypo, test[1], lower_mean_price_ratio)

run_ttest()

FileNotFoundError: [Errno 2] File City_Zhvi_AllHomes.csv does not exist: 'City_Zhvi_AllHomes.csv'